# 04 - EDA: Player Progression

**Purpose**: Understand trophy progression patterns and player skill development.

**Key Questions**:
- Where do players hit trophy "walls"?
- How do deck characteristics change with trophy level?
- Is there trophy inflation/deflation?
- What separates winners from losers at different skill levels?

**Potential Insights**:
- Identify key trophy milestones (4000, 5000, 6000)
- Show how successful decks evolve with skill
- Analyze trophy risk at different levels

In [ ]:
import sys, os, duckdb, pandas as pd, numpy as np
import matplotlib.pyplot as plt, seaborn as sns

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, os.path.join(PROJECT_ROOT, 'src'))

# Use Parquet if available (faster), fallback to CSV
DATA_PATH = os.path.join(PROJECT_ROOT, 'battles.parquet')
if not os.path.exists(DATA_PATH):
    DATA_PATH = os.path.join(PROJECT_ROOT, 'battles.csv')

from duckdb_utils import get_connection, create_battles_view, query_to_df
from visualization import setup_presentation_style, plot_distribution
from feature_engineering import create_trophy_bracket_features

con = get_connection()
create_battles_view(con, DATA_PATH)
setup_presentation_style()

## 1. Trophy Distribution

Visualize where players cluster

In [ ]:
# Query trophy distribution
trophy_dist_query = """
SELECT 
    "average.startingTrophies" as starting_trophies
FROM battles
WHERE "average.startingTrophies" IS NOT NULL
"""

trophy_dist = query_to_df(con, trophy_dist_query)

# Create histogram with KDE
fig = plot_distribution(
    trophy_dist['starting_trophies'],
    'Trophy Distribution: Where Players Cluster',
    'Starting Trophies',
    bins=100,
    kde=True
)

# Identify peaks (trophy walls)
# Use binning to find concentration points
trophy_bins = pd.cut(trophy_dist['starting_trophies'], bins=50)
bin_counts = trophy_bins.value_counts().sort_index()

print("\nTop 5 trophy concentration points (walls):")
top_bins = bin_counts.nlargest(5)
for interval, count in top_bins.items():
    print(f"  {interval}: {count:,} battles")

plt.show()

# Key insights
print("\n📊 Key Insights:")
print(f"  - Mean trophy level: {trophy_dist['starting_trophies'].mean():.0f}")
print(f"  - Median trophy level: {trophy_dist['starting_trophies'].median():.0f}")
print(f"  - Trophy range: {trophy_dist['starting_trophies'].min():.0f} to {trophy_dist['starting_trophies'].max():.0f}")

## 2. Trophy Change Analysis

In [ ]:
# Analyze trophy change patterns by trophy bracket
trophy_change_query = """
SELECT 
    FLOOR("winner.startingTrophies" / 500) * 500 as trophy_bracket,
    AVG("winner.trophyChange") as avg_gain,
    STDDEV("winner.trophyChange") as std_gain,
    MIN("winner.trophyChange") as min_gain,
    MAX("winner.trophyChange") as max_gain,
    COUNT(*) as battle_count
FROM battles
WHERE "winner.trophyChange" IS NOT NULL
    AND "winner.startingTrophies" IS NOT NULL
GROUP BY trophy_bracket
ORDER BY trophy_bracket
"""

trophy_change = query_to_df(con, trophy_change_query)

# Visualize trophy gain by bracket
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Average trophy gain
ax1.bar(trophy_change['trophy_bracket'], trophy_change['avg_gain'], 
        color='steelblue', edgecolor='black', alpha=0.7)
ax1.set_xlabel('Trophy Bracket', fontsize=14)
ax1.set_ylabel('Average Trophy Gain', fontsize=14)
ax1.set_title('Average Trophy Gain by Bracket', fontsize=16, fontweight='bold')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(axis='y', alpha=0.3)

# Standard deviation (risk)
ax2.bar(trophy_change['trophy_bracket'], trophy_change['std_gain'], 
        color='coral', edgecolor='black', alpha=0.7)
ax2.set_xlabel('Trophy Bracket', fontsize=14)
ax2.set_ylabel('Trophy Gain Volatility (Std Dev)', fontsize=14)
ax2.set_title('Trophy Risk by Bracket', fontsize=16, fontweight='bold')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Display summary table
print("\n📊 Trophy Change Analysis by Bracket:")
print(trophy_change.to_string(index=False))

print("\n💡 Key Insights:")
print(f"  - Highest avg gain: {trophy_change.loc[trophy_change['avg_gain'].idxmax(), 'trophy_bracket']:.0f} bracket ({trophy_change['avg_gain'].max():.2f} trophies)")
print(f"  - Most volatile: {trophy_change.loc[trophy_change['std_gain'].idxmax(), 'trophy_bracket']:.0f} bracket (±{trophy_change['std_gain'].max():.2f} trophies)")
print(f"  - Most battles: {trophy_change.loc[trophy_change['battle_count'].idxmax(), 'trophy_bracket']:.0f} bracket ({trophy_change['battle_count'].max():,} battles)")

## 3. Deck Evolution by Trophy Level

In [ ]:
# Analyze how deck characteristics evolve with trophy level
deck_evolution_query = """
WITH trophy_brackets AS (
    SELECT 
        CASE 
            WHEN "winner.startingTrophies" < 1000 THEN '0-1k'
            WHEN "winner.startingTrophies" < 2000 THEN '1k-2k'
            WHEN "winner.startingTrophies" < 3000 THEN '2k-3k'
            WHEN "winner.startingTrophies" < 4000 THEN '3k-4k'
            WHEN "winner.startingTrophies" < 5000 THEN '4k-5k'
            WHEN "winner.startingTrophies" < 6000 THEN '5k-6k'
            WHEN "winner.startingTrophies" < 7000 THEN '6k-7k'
            ELSE '7k+'
        END as bracket,
        "winner.elixir.average" as elixir_avg,
        "winner.rarity.legendary.count" as legendary_count,
        "winner.spell.count" as spell_count,
        "winner.troop.count" as troop_count,
        "winner.structure.count" as structure_count
    FROM battles
    WHERE "winner.startingTrophies" IS NOT NULL
        AND "winner.elixir.average" IS NOT NULL
)
SELECT 
    bracket,
    AVG(elixir_avg) as avg_elixir,
    AVG(legendary_count) as avg_legendary,
    AVG(spell_count) as avg_spells,
    AVG(troop_count) as avg_troops,
    AVG(structure_count) as avg_structures,
    COUNT(*) as battle_count
FROM trophy_brackets
GROUP BY bracket
ORDER BY 
    CASE bracket
        WHEN '0-1k' THEN 1
        WHEN '1k-2k' THEN 2
        WHEN '2k-3k' THEN 3
        WHEN '3k-4k' THEN 4
        WHEN '4k-5k' THEN 5
        WHEN '5k-6k' THEN 6
        WHEN '6k-7k' THEN 7
        ELSE 8
    END
"""

deck_evolution = query_to_df(con, deck_evolution_query)

# Create multi-panel visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Elixir cost evolution
axes[0, 0].plot(deck_evolution['bracket'], deck_evolution['avg_elixir'], 
                marker='o', linewidth=2, markersize=8, color='steelblue')
axes[0, 0].set_title('Average Elixir Cost by Trophy Level', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Trophy Bracket')
axes[0, 0].set_ylabel('Avg Elixir Cost')
axes[0, 0].grid(alpha=0.3)
axes[0, 0].tick_params(axis='x', rotation=45)

# Legendary usage
axes[0, 1].plot(deck_evolution['bracket'], deck_evolution['avg_legendary'], 
                marker='s', linewidth=2, markersize=8, color='gold')
axes[0, 1].set_title('Legendary Card Usage by Trophy Level', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Trophy Bracket')
axes[0, 1].set_ylabel('Avg Legendary Cards per Deck')
axes[0, 1].grid(alpha=0.3)
axes[0, 1].tick_params(axis='x', rotation=45)

# Spell/Troop/Structure composition
axes[1, 0].plot(deck_evolution['bracket'], deck_evolution['avg_spells'], 
                marker='^', linewidth=2, markersize=8, label='Spells', color='purple')
axes[1, 0].plot(deck_evolution['bracket'], deck_evolution['avg_troops'], 
                marker='v', linewidth=2, markersize=8, label='Troops', color='red')
axes[1, 0].plot(deck_evolution['bracket'], deck_evolution['avg_structures'], 
                marker='D', linewidth=2, markersize=8, label='Structures', color='green')
axes[1, 0].set_title('Deck Composition by Trophy Level', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Trophy Bracket')
axes[1, 0].set_ylabel('Avg Cards per Deck')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)
axes[1, 0].tick_params(axis='x', rotation=45)

# Battle count distribution
axes[1, 1].bar(deck_evolution['bracket'], deck_evolution['battle_count'], 
               color='coral', edgecolor='black', alpha=0.7)
axes[1, 1].set_title('Battle Distribution by Trophy Level', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Trophy Bracket')
axes[1, 1].set_ylabel('Number of Battles')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Display summary table
print("\n📊 Deck Evolution by Trophy Level:")
print(deck_evolution.to_string(index=False))

print("\n💡 Key Insights:")
print(f"  - Elixir trend: {deck_evolution['avg_elixir'].iloc[0]:.2f} (low) → {deck_evolution['avg_elixir'].iloc[-1]:.2f} (high)")
print(f"  - Legendary usage: {deck_evolution['avg_legendary'].iloc[0]:.2f} (low) → {deck_evolution['avg_legendary'].iloc[-1]:.2f} (high)")
print(f"  - Spell usage: {deck_evolution['avg_spells'].iloc[0]:.2f} (low) → {deck_evolution['avg_spells'].iloc[-1]:.2f} (high)")

## 4. Matchup Fairness

In [ ]:
# Analyze trophy differential between winner and loser
matchup_fairness_query = """
SELECT 
    ("winner.startingTrophies" - "loser.startingTrophies") as trophy_diff,
    "winner.startingTrophies" as winner_trophies,
    "loser.startingTrophies" as loser_trophies,
    "winner.trophyChange" as trophy_change
FROM battles
WHERE "winner.startingTrophies" IS NOT NULL
    AND "loser.startingTrophies" IS NOT NULL
    AND ABS("winner.startingTrophies" - "loser.startingTrophies") < 2000  -- Filter extreme outliers
LIMIT 100000  -- Sample for performance
"""

matchup_data = query_to_df(con, matchup_fairness_query)

# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Distribution of trophy differential
axes[0, 0].hist(matchup_data['trophy_diff'], bins=100, color='steelblue', edgecolor='black', alpha=0.7)
axes[0, 0].axvline(0, color='red', linestyle='--', linewidth=2, label='Even Match')
axes[0, 0].set_xlabel('Trophy Differential (Winner - Loser)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Matchup Fairness: Trophy Differential Distribution', fontsize=14, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Scatter: Trophy diff vs Trophy change
axes[0, 1].scatter(matchup_data['trophy_diff'], matchup_data['trophy_change'], 
                   alpha=0.3, s=1, color='darkblue')
axes[0, 1].axvline(0, color='red', linestyle='--', linewidth=1, alpha=0.5)
axes[0, 1].set_xlabel('Trophy Differential (Winner - Loser)')
axes[0, 1].set_ylabel('Trophy Change for Winner')
axes[0, 1].set_title('Trophy Diff vs Trophy Reward', fontsize=14, fontweight='bold')
axes[0, 1].grid(alpha=0.3)

# Box plot: Trophy change by differential category
matchup_data['diff_category'] = pd.cut(matchup_data['trophy_diff'], 
                                        bins=[-1000, -200, -50, 50, 200, 1000],
                                        labels=['Underdog (<-200)', 'Slight Underdog', 
                                               'Even Match', 'Slight Favorite', 'Heavy Favorite (>200)'])
axes[1, 0].boxplot([matchup_data[matchup_data['diff_category'] == cat]['trophy_change'].dropna() 
                     for cat in matchup_data['diff_category'].cat.categories],
                   labels=matchup_data['diff_category'].cat.categories,
                   patch_artist=True)
axes[1, 0].set_ylabel('Trophy Change for Winner')
axes[1, 0].set_title('Trophy Reward by Match Fairness', fontsize=14, fontweight='bold')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(axis='y', alpha=0.3)

# Underdog win rate analysis
axes[1, 1].axis('off')
underdog_wins = (matchup_data['trophy_diff'] < 0).sum()
total_matches = len(matchup_data)
underdog_rate = (underdog_wins / total_matches) * 100

stats_text = f"""
📊 Matchup Fairness Statistics:

Trophy Differential:
  • Mean: {matchup_data['trophy_diff'].mean():.1f} trophies
  • Median: {matchup_data['trophy_diff'].median():.1f} trophies
  • Std Dev: {matchup_data['trophy_diff'].std():.1f} trophies

Underdog Performance:
  • Underdog win rate: {underdog_rate:.2f}%
  • Total underdog wins: {underdog_wins:,}
  • Close matches (±50): {((matchup_data['trophy_diff'].abs() <= 50).sum() / total_matches * 100):.1f}%

Trophy Rewards:
  • Even match reward: {matchup_data[matchup_data['trophy_diff'].abs() <= 50]['trophy_change'].mean():.2f}
  • Underdog win reward: {matchup_data[matchup_data['trophy_diff'] < -100]['trophy_change'].mean():.2f}
  • Favorite win reward: {matchup_data[matchup_data['trophy_diff'] > 100]['trophy_change'].mean():.2f}
"""
axes[1, 1].text(0.1, 0.5, stats_text, fontsize=12, verticalalignment='center',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print("\n💡 Key Insights:")
print(f"  - {underdog_rate:.1f}% of winners had fewer trophies than their opponent (underdog wins)")
print(f"  - Average trophy differential: {matchup_data['trophy_diff'].mean():.1f} trophies")
print(f"  - {((matchup_data['trophy_diff'].abs() <= 50).sum() / total_matches * 100):.1f}% of matches are within ±50 trophies (fair matches)")

## Key Insights

**Document progression patterns for presentation**